# Example of Plots using GEOtoPy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from mhpc_project.matsch_b2 import FullModel
from geotopy.utils import date_parser, comparison_plot

In [ ]:
targets = [
    ('rainfall_amount', 'mm', 'Rainfall Amount'),
    ('wind_speed', 'm/s', 'Wind Speed'),
    ('relative_humidity', None, 'Relative Humidity'),
     ('air_temperature', 'C', 'Air Temperature'),
    ('surface_downwelling_shortwave_flux', 'W/m2', 'Downwelling Shortwave Flux'),
    ('soil_moisture_content_50', None, 'Soil Moisture at 50mm'),
    ('soil_moisture_content_200', None, 'Soil Moisture at 200mm'),
    ('latent_heat_flux_in_air', 'W/m2', 'Latent Heat Flux'),
    ('sensible_heat_flux_in_air', 'W/m2', 'Sensible Heat Flux')
]

observations = pd.read_csv('../data/Matsch B2/obs.csv',
                           na_values=['-9999', '-99.99'],
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)
observations.index.rename('datetime', inplace=True)
model = FullModel('../data/Matsch B2/geotop')

In [ ]:
simulation = model()
for name, unit, desc in targets:
    display(Markdown(f"# {desc}"))
    comparison_plot(observations[name],
                    simulation[name],
                    desc=desc,
                    unit=unit)
    plt.show()